## Fedezd fel a K-Means klaszterezést R és Tidy adatelvek segítségével.

### [**Előadás előtti kvíz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Ebben a leckében megtanulod, hogyan hozz létre klasztereket a Tidymodels csomag és az R ökoszisztéma más csomagjai (hívjuk őket barátoknak 🧑‍🤝‍🧑) segítségével, valamint az előzőekben importált nigériai zenei adathalmazzal. Áttekintjük a K-Means klaszterezés alapjait. Ne feledd, ahogy az előző leckében tanultad, számos módja van a klaszterekkel való munkának, és az alkalmazott módszer az adataidtól függ. Mi a K-Means-t fogjuk kipróbálni, mivel ez a leggyakoribb klaszterezési technika. Kezdjük is!

Fogalmak, amelyeket megismerhetsz:

-   Sziluett pontszám

-   Könyökmódszer

-   Inercia

-   Variancia

### **Bevezetés**

A [K-Means klaszterezés](https://wikipedia.org/wiki/K-means_clustering) a jelfeldolgozás területéből származó módszer. Arra használják, hogy az adatokat `k klaszterekre` osszák és csoportosítsák a jellemzőik hasonlósága alapján.

A klaszterek [Voronoi diagramokként](https://wikipedia.org/wiki/Voronoi_diagram) is megjeleníthetők, amelyek tartalmaznak egy pontot (vagy 'magot') és annak megfelelő régióját.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografika: Jen Looper</figcaption>

A K-Means klaszterezés lépései a következők:

1.  Az adatelemző meghatározza a létrehozandó klaszterek kívánt számát.

2.  Ezután az algoritmus véletlenszerűen kiválaszt K megfigyelést az adathalmazból, hogy ezek szolgáljanak a klaszterek kezdeti középpontjaként (azaz centroidként).

3.  Ezután a fennmaradó megfigyeléseket hozzárendelik a legközelebbi centroidhoz.

4.  Ezután minden klaszter új átlagát kiszámítják, és a centroidot az átlaghoz mozgatják.

5.  Most, hogy a középpontokat újraszámították, minden megfigyelést újra ellenőriznek, hogy közelebb lehet-e egy másik klaszterhez. Az összes objektumot újra hozzárendelik az új klaszterátlagok alapján. A klaszter-hozzárendelési és centroid-frissítési lépéseket iteratívan ismétlik, amíg a klaszter-hozzárendelések nem változnak tovább (azaz amikor konvergencia történik). Az algoritmus általában akkor áll le, amikor minden új iteráció elhanyagolható centroid-mozgást eredményez, és a klaszterek statikussá válnak.

<div>

> Ne feledd, hogy a kezdeti k megfigyelések véletlenszerű kiválasztása miatt, amelyek a kezdő centroidként szolgálnak, kissé eltérő eredményeket kaphatunk minden alkalommal, amikor alkalmazzuk az eljárást. Emiatt a legtöbb algoritmus több *véletlen kezdést* használ, és azt az iterációt választja, amely a legalacsonyabb WCSS-t eredményezi. Ezért erősen ajánlott, hogy mindig több *nstart* értékkel futtasd a K-Means-t, hogy elkerüld a *nem kívánt lokális optimumot.*

</div>

Ez a rövid animáció Allison Horst [műalkotásával](https://github.com/allisonhorst/stats-illustrations) magyarázza a klaszterezési folyamatot:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Műalkotás: @allison_horst</figcaption>

Egy alapvető kérdés, amely a klaszterezés során felmerül: honnan tudod, hány klaszterre kell osztanod az adataidat? A K-Means egyik hátránya, hogy meg kell határoznod `k`-t, azaz a `centroidok` számát. Szerencsére a `könyökmódszer` segít egy jó kiindulási érték becslésében `k` számára. Mindjárt ki is próbálod.

### 

**Előfeltétel**

Ott folytatjuk, ahol az [előző leckében](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) abbahagytuk, ahol elemeztük az adathalmazt, számos vizualizációt készítettünk, és az adathalmazt érdekes megfigyelésekre szűrtük. Mindenképpen nézd meg!

Néhány csomagra szükségünk lesz, hogy belevágjunk ebbe a modulba. Telepítheted őket így: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternatívaként az alábbi szkript ellenőrzi, hogy megvannak-e a szükséges csomagok a modul befejezéséhez, és telepíti őket, ha hiányoznak.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Kezdjünk bele!

## 1. Egy tánc az adatokkal: Szűkítsük le a 3 legnépszerűbb zenei műfajra

Ez egy összefoglaló arról, amit az előző leckében csináltunk. Vágjunk bele az adatok elemzésébe!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Ez jól sikerült!

## 2. További adatfeltárás.

Mennyire tiszta ez az adat? Nézzük meg a szélsőséges értékeket boxplotok segítségével. Koncentráljunk azokra a numerikus oszlopokra, amelyekben kevesebb a szélsőséges érték (bár ezeket ki is lehet tisztítani). A boxplotok megmutatják az adatok tartományát, és segítenek eldönteni, mely oszlopokat érdemes használni. Fontos megjegyezni, hogy a boxplotok nem mutatják a szórást, ami a jól klaszterezhető adatok egyik fontos eleme. További információért lásd [ezt a vitát](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot).

A [boxplotok](https://en.wikipedia.org/wiki/Box_plot) grafikus módon ábrázolják a `numerikus` adatok eloszlását, ezért kezdjük azzal, hogy *kiválasztjuk* az összes numerikus oszlopot a népszerű zenei műfajokkal együtt.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Látod, milyen egyszerűvé teszi a `where` kiválasztási segéd az ilyen feladatokat 💁? Fedezz fel további hasonló funkciókat [itt](https://tidyselect.r-lib.org/).

Mivel minden numerikus jellemzőhöz boxplotot fogunk készíteni, és el szeretnénk kerülni a ciklusok használatát, alakítsuk át az adatainkat egy *hosszabb* formátumra, amely lehetővé teszi számunkra, hogy kihasználjuk a `facets` előnyeit - olyan aládiagramokat, amelyek mindegyike az adatok egy-egy részhalmazát jeleníti meg.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Sokkal hosszabb! Most itt az ideje néhány `ggplot`-nak! Szóval, melyik `geom`-ot fogjuk használni?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Könnyű-gg!

Most már láthatjuk, hogy ez az adat kissé zajos: ha minden oszlopot boxplotként megfigyelünk, láthatók a szélsőértékek. Át lehetne nézni az adathalmazt, és eltávolítani ezeket a szélsőértékeket, de ez eléggé lecsökkentené az adatokat.

Egyelőre válasszuk ki, mely oszlopokat fogjuk használni a klaszterezési gyakorlatunkhoz. Válasszuk ki a hasonló tartományú numerikus oszlopokat. Az `artist_top_genre` oszlopot numerikusként is kódolhatnánk, de most elhagyjuk.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. K-means klaszterezés számítása R-ben

Az R-ben a beépített `kmeans` függvénnyel számíthatjuk ki a k-means klaszterezést, lásd `help("kmeans()")`. A `kmeans()` függvény elsődleges argumentumként egy adatkeretet fogad el, amelynek minden oszlopa numerikus.

A k-means klaszterezés használatakor az első lépés az, hogy megadjuk a klaszterek számát (k), amelyeket a végső megoldásban létrehozunk. Tudjuk, hogy az adatállományból 3 zenei műfajt különítettünk el, így próbáljuk meg a 3-at:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


A kmeans objektum több információt tartalmaz, amelyet jól elmagyaráz a `help("kmeans()")`. Most koncentráljunk néhányra. Láthatjuk, hogy az adatokat 3 klaszterbe csoportosították, amelyek méretei: 65, 110, 111. Az eredmény tartalmazza a klaszterek középpontjait (átlagait) is a 3 csoport esetében az 5 változón keresztül.

A klaszterezési vektor az egyes megfigyelések klaszterhez való hozzárendelését jelöli. Használjuk az `augment` függvényt, hogy hozzáadjuk a klaszterhez való hozzárendelést az eredeti adatállományhoz.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Tökéletes, most éppen 3 csoportra osztottuk az adathalmazunkat. De mennyire jó a klaszterezésünk 🤷? Nézzük meg a `Silhouette pontszámot`.

### **Silhouette pontszám**

A [Silhouette elemzés](https://en.wikipedia.org/wiki/Silhouette_(clustering)) segítségével tanulmányozhatjuk az eredményül kapott klaszterek közötti távolságot. Ez a pontszám -1 és 1 között változik, és ha a pontszám közel van az 1-hez, akkor a klaszter sűrű és jól elkülönül a többi klasztertől. A 0-hoz közeli érték átfedő klasztereket jelez, ahol a minták nagyon közel vannak a szomszédos klaszterek döntési határához. [forrás](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Az átlagos silhouette módszer kiszámítja a megfigyelések átlagos silhouette értékét különböző *k* értékekre. Egy magas átlagos silhouette pontszám jó klaszterezést jelez.

A `silhouette` függvény a cluster csomagban használható az átlagos silhouette szélesség kiszámítására.

> A silhouette bármilyen [távolság](https://en.wikipedia.org/wiki/Distance "Distance") metrikával kiszámítható, például az [Euklideszi távolság](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") vagy a [Manhattani távolság](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") segítségével, amelyeket az [előző leckében](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) tárgyaltunk.


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Pontszámunk **0,549**, tehát pontosan középen van. Ez arra utal, hogy adataink nem különösebben alkalmasak az ilyen típusú klaszterezésre. Nézzük meg, hogy vizuálisan meg tudjuk-e erősíteni ezt a feltételezést. A [factoextra csomag](https://rpkgs.datanovia.com/factoextra/index.html) olyan funkciókat biztosít (`fviz_cluster()`), amelyekkel a klaszterezés vizualizálható.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Az átfedés a klaszterek között azt jelzi, hogy adataink nem különösebben alkalmasak erre a fajta klaszterezésre, de folytassuk.

## 4. Az optimális klaszterek meghatározása

Egy alapvető kérdés, amely gyakran felmerül a K-Means klaszterezés során, a következő: ha nincsenek ismert osztálycímkék, honnan tudjuk, hány klaszterre kell osztani az adatokat?

Egy lehetséges módszer az, hogy egy adatmintát használunk, és `létrehozunk egy sor klaszterezési modellt` növekvő klaszterszámmal (például 1-től 10-ig), majd kiértékeljük a klaszterezési metrikákat, mint például a **Silhouette pontszám.**

Határozzuk meg az optimális klaszterszámot úgy, hogy különböző *k* értékekre kiszámítjuk a klaszterezési algoritmust, és kiértékeljük a **klaszteren belüli négyzetösszeget** (WCSS). A klaszteren belüli négyzetösszeg (WCSS) az összetartozóságot méri, és azt szeretnénk, hogy ez minél kisebb legyen, mivel az alacsonyabb értékek azt jelentik, hogy az adatpontok közelebb vannak egymáshoz.

Vizsgáljuk meg, hogyan hatnak a különböző `k` választások, 1-től 10-ig, erre a klaszterezésre.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Most, hogy megvan az összesített klaszteren belüli négyzetösszeg (tot.withinss) minden klaszterezési algoritmus esetében *k* középponttal, az [könyökmódszert](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) használjuk az optimális klaszterszám meghatározására. A módszer abból áll, hogy ábrázoljuk a WCSS-t a klaszterek számának függvényében, és kiválasztjuk a [görbe könyökét](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") a használni kívánt klaszterszámként.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


A grafikon jelentős csökkenést mutat a WCSS-ben (tehát nagyobb *szorosságot*), ahogy a klaszterek száma egytől kettőig növekszik, majd további észrevehető csökkenést két és három klaszter között. Ezt követően a csökkenés kevésbé kifejezett, ami egy `könyök` 💪-t eredményez a grafikonon körülbelül három klaszternél. Ez jó jelzés arra, hogy két-három viszonylag jól elkülönített klaszter található az adatpontok között.

Most folytathatjuk a klaszterezési modell kinyerését, ahol `k = 3`:

> `pull()`: egyetlen oszlop kinyerésére használható
>
> `pluck()`: adatszerkezetek, például listák indexelésére használható


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Szuper! Nézzük meg a kapott klasztereket. Mit szólnál egy kis interaktivitáshoz a `plotly` segítségével?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Talán azt vártuk volna, hogy minden klaszter (különböző színekkel jelölve) eltérő műfajokat képviseljen (különböző alakzatokkal jelölve).

Nézzük meg a modell pontosságát.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Ennek a modellnek a pontossága nem rossz, de nem is kiemelkedő. Lehetséges, hogy az adatok nem igazán alkalmasak a K-Means klaszterezéshez. Az adatok túl kiegyensúlyozatlanok, kevéssé korreláltak, és túl nagy a variancia az oszlopértékek között ahhoz, hogy jól klaszterezhetők legyenek. Valójában a kialakuló klaszterek valószínűleg erősen befolyásoltak vagy torzítottak az általunk fentebb meghatározott három műfajkategória által.

Ennek ellenére ez egy igazán tanulságos folyamat volt!

A Scikit-learn dokumentációjában látható, hogy egy ilyen modell, ahol a klaszterek nem túl jól elkülönültek, "variancia" problémával küzd:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografika a Scikit-learn-től</figcaption>



## **Variancia**

A variancia definíciója: "az átlagtól való négyzetes eltérések átlaga" [forrás](https://www.mathsisfun.com/data/standard-deviation.html). Ebben a klaszterezési problémában arra utal, hogy az adataink számai hajlamosak túlságosan eltérni az átlagtól.

✅ Ez egy remek alkalom arra, hogy átgondold, milyen módokon lehetne ezt a problémát orvosolni. Finomítsd az adatokat? Használj más oszlopokat? Próbálj ki egy másik algoritmust? Tipp: Próbáld meg [normalizálni az adatokat](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) és tesztelj más oszlopokat.

> Próbáld ki ezt a '[variancia kalkulátort](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', hogy jobban megértsd a koncepciót.

------------------------------------------------------------------------

## **🚀Kihívás**

Tölts el egy kis időt ezzel a jegyzettel, és finomítsd a paramétereket. Tudod javítani a modell pontosságát az adatok tisztításával (például a kiugró értékek eltávolításával)? Használhatsz súlyokat, hogy bizonyos adatminták nagyobb súlyt kapjanak. Mit tehetsz még, hogy jobb klasztereket hozz létre?

Tipp: Próbáld meg skálázni az adatokat. A jegyzetben van kommentált kód, amely hozzáadja a standard skálázást, hogy az adat oszlopai jobban hasonlítsanak egymásra tartományuk tekintetében. Meg fogod látni, hogy bár a sziluett pontszám csökken, az "elbow" grafikon görbülete kisimul. Ez azért van, mert ha az adatokat nem skálázod, akkor a kisebb varianciájú adatok nagyobb súlyt kapnak. Olvass többet erről a problémáról [itt](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Utólagos kvíz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Áttekintés és önálló tanulás**

-   Nézz meg egy K-Means szimulátort [például ezt](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Ezzel az eszközzel vizualizálhatod a mintapontokat és meghatározhatod a centroidokat. Szerkesztheted az adatok véletlenszerűségét, a klaszterek számát és a centroidok számát. Segít ez abban, hogy jobban megértsd, hogyan csoportosíthatók az adatok?

-   Nézd meg [ezt a K-Means kézikönyvet](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) a Stanfordtól.

Szeretnéd kipróbálni az újonnan megszerzett klaszterezési készségeidet olyan adathalmazokon, amelyek jól illeszkednek a K-Means klaszterezéshez? Nézd meg:

-   [Klaszterezési modellek tanítása és értékelése](https://rpubs.com/eR_ic/clustering) Tidymodels és társai segítségével

-   [K-means klaszterelemzés](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-means klaszterezés rendezett adatok elveivel](https://www.tidymodels.org/learn/statistics/k-means/)

## **Feladat**

[Próbálj ki különböző klaszterezési módszereket](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## KÖSZÖNET:

[Jen Looper](https://www.twitter.com/jenlooper) az eredeti Python verzió elkészítéséért ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) az elképesztő illusztrációkért, amelyek barátságosabbá és vonzóbbá teszik az R-t. További illusztrációkat találhatsz a [galériájában](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Jó tanulást kíván,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Illusztráció @allison_horst-tól</figcaption>



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális, emberi fordítást igénybe venni. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
